In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pickle
from initialize import initialState
from util import action_set, to_action
from write_yed_template import create_gml
from interactions import common
from action import Action, Source

In [24]:
import reference_model

In [3]:
def compile_expression(actions, true_interactions, predefined):
    rule = set()
    for action in actions:
        correct = action in true_interactions and action not in predefined
        predefd = action in predefined

        if correct:
            template = '\colorbox{{CornflowerBlue}}{{{item}}}'
        elif predefd:
            template = '\colorbox{{Melon}}{{{item}}}'
        else:
            template = '{item}'

        c = set()
        for src, span in action.srcs:
            cell_idx = '{i}' if span == 'i' else '{i \pm 1}'
            if src[0] == '~':
                src = src[1:]
                item = f"not ${src}_{cell_idx}$"
            else:
                item = f"${src}_{cell_idx}$"
            c.add(item)
        compiled_action = ' and '.join(c)
        formatted_action = template.format(item=compiled_action)

        rule.add(formatted_action)
        
    compiled_expression = ' or '.join(rule)
    return compiled_expression    


def print_rules(model, true_actions, predef_actions):
    for dst, (activators, inhibitors) in model.rules.items():
        sdst, cell = dst
        if cell == 1:
            inhibitor_rule = compile_expression(
                [to_action(src=inhibitor, dst=dst, itype='i') for inhibitor in inhibitors], 
                true_actions, predef_actions)
            activator_rule = compile_expression(
                [to_action(src=activator, dst=dst, itype='a') for activator in activators], 
                true_actions, predef_actions)

            if activator_rule and inhibitor_rule:
                rule = f'[{activator_rule}] and not [{inhibitor_rule}]'
            elif activator_rule:
                rule = f'[{activator_rule}]'
            elif inhibitor_rule:
                rule = f'not [{inhibitor_rule}]'
            else:
                rule = False
            print(f'${sdst}_i$ = {rule}\n')

In [4]:
data_dir = '/home/bglaze/segment_polarity/data/'

In [5]:
tmstp = '20210415-1427'

In [6]:
cluster_models = pickle.load(open(f'{data_dir}/models/best_models/best_models_clusters_tmstp-{tmstp}.pickle','rb'))

In [7]:
predefined,_ = initialState()

predef_actions = action_set(predefined.rules)

In [25]:
create_gml(reference_model.actions, true=reference_model.actions, predefined=dict(),
           outfile=f'{data_dir}/figures/reference_segment_polarity_model.gml')

In [26]:
create_gml(predef_actions, true=predef_actions, predefined=dict(),
           outfile=f'{data_dir}/figures/predefined_actions.gml')

In [22]:
cluster_inters = {}
cluster_jaccards = {}
for cluster, models in cluster_models.items():
    inters = []
    cluster_inters[cluster] = inters
    jaccards = []
    cluster_jaccards[cluster] = jaccards
    for model in models:
        inter = len(action_set(model) & reference_model.actions - predef_actions)
        union =  len(action_set(model) | reference_model.actions - predef_actions)
        jaccards.append(inter/union)
        inters.append(inter)

### Whole cluster statistics for intersection with Reka's model

In [11]:
from statistics import mean, median

In [12]:
print("Cluster:", "Mean", 'Median', "Max", "Count")
for i,c_inters in cluster_inters.items():
    print(f'{i:7d}: {mean(c_inters):2.2f} {median(c_inters):6.0f} {max(c_inters):3.0f} {len(c_inters):5.0f}')

Cluster: Mean Median Max Count
      0: 4.80      5   8  5321
      1: 6.03      6   9  6566
      2: 6.32      6   9  1654
      3: 3.23      3   6  2057
      4: 6.87      7   9  5673
      5: 6.49      6   9  4805
      6: 3.30      3   6  6710
      7: 8.68      9  11  5493
      8: 5.22      5   8  2277
      9: 6.90      7  10  6413
     10: 8.27      8  11  3031


In [23]:
print("Cluster:", "Mean", 'Median', "Max", "Count")
for i,c_jaccards in cluster_jaccards.items():
    print(f'{i:7d}: {mean(c_jaccards):2.2f} {median(c_jaccards):6.2f} {max(c_jaccards):3.2f} {len(c_jaccards):5.0f}')

Cluster: Mean Median Max Count
      0: 0.10   0.11 0.18  5321
      1: 0.14   0.14 0.23  6566
      2: 0.17   0.16 0.24  1654
      3: 0.08   0.08 0.16  2057
      4: 0.17   0.17 0.26  5673
      5: 0.15   0.15 0.24  4805
      6: 0.07   0.07 0.15  6710
      7: 0.22   0.22 0.33  5493
      8: 0.12   0.12 0.18  2277
      9: 0.17   0.17 0.26  6413
     10: 0.18   0.18 0.24  3031


### Cluster common model intersection with Reka's model

In [13]:
common_models = dict()
print('Cluster   Common Model Intersection')
for i,models in cluster_models.items():
    if i!=-1: # exclude the "unassigned" points. DBSCAN labels these -1
        threshold = .9
        actions = set([action[0] for action, count in common(models).items() if count > threshold])
        
        intersection = actions & reference_model.actions - predef_actions
        common_models[i] = intersection
        print(f'{i:7d} {len(intersection):3d}')

Cluster   Common Model Intersection
      0   3
      1   5
      2   6
      3   3
      4   6
      5   6
      6   3
      7   8
      8   5
      9   6
     10   7


In [14]:
import initialize
from tree_search import add_to_rules

In [17]:
for i,models in cluster_models.items():
    threshold = .9
    
    actions = {action[0] for action, count in common(models).items() if count > threshold}

    create_gml(actions, true=reference_model.actions, predefined=predef_actions,
               outfile=f'{data_dir}/figures/common_interactions_tmstp-{tmstp}_cluster-{i}.gml')
    create_gml(actions, true=reference_model.actions, predefined=predef_actions,
               outfile=f'{data_dir}/figures/false_positive_interactions_tmstp-{tmstp}_cluster-{i}.gml',
               highlighted='false_positive')
    create_gml(reference_model.actions, true=actions, predefined=predef_actions,
               outfile=f'{data_dir}/figures/false_negative_interactions_tmstp-{tmstp}_cluster-{i}.gml',
               highlighted='false_positive')

    
#     print(f'Cluster {i}\n')
#     print_rules(common_model, reference_model.actions, predef_actions)
#     print('\bigskip')`

In [30]:
for i,models in cluster_models.items():
    actions = {action[0] for action, count in common(models).items() if count > threshold}

    print(f'Cluster {i}\n')
    print_rules(actions, reference_model.actions, predef_actions)


Cluster 0



AttributeError: 'set' object has no attribute 'rules'

In [31]:
actions

{Action(srcs=(('CI', 'i'),), dst='CIA', type='a'),
 Action(srcs=(('CI', 'i'),), dst='CIR', type='a'),
 Action(srcs=(('CI', 'i'),), dst='en', type='i'),
 Action(srcs=(('CI', 'i'), ('SMO', 'i')), dst='ptc', type='a'),
 Action(srcs=(('CIA', 'i'),), dst='SMO', type='a'),
 Action(srcs=(('CIA', 'i'), ('SLP', 'i')), dst='CI', type='a'),
 Action(srcs=(('CIA', 'i'), ('SLP', 'i')), dst='wg', type='a'),
 Action(srcs=(('CIR', 'i'),), dst='CIA', type='i'),
 Action(srcs=(('CIR', 'i'),), dst='PH', type='i'),
 Action(srcs=(('CIR', 'i'),), dst='WG', type='i'),
 Action(srcs=(('EN', 'i'),), dst='CI', type='i'),
 Action(srcs=(('EN', 'i'),), dst='HH', type='a'),
 Action(srcs=(('EN', 'i'),), dst='ci', type='i'),
 Action(srcs=(('EN', 'i'),), dst='hh', type='a'),
 Action(srcs=(('HH', 'e'),), dst='CIR', type='i'),
 Action(srcs=(('HH', 'i'),), dst='PH', type='i'),
 Action(srcs=(('PH', 'i'),), dst='PTC', type='a'),
 Action(srcs=(('PTC', 'i'), ('HH', 'e')), dst='PH', type='a'),
 Action(srcs=(('PTC', 'i'), ('~HH',